This code creates files with the history for the prey and predator for every parameter configuration.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import integrate
import json

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Matching Factor

## Get Machting Factor from Reshuffle

In [ ]:
def read_json(index):
    with open(f'/content/drive/MyDrive/NaCo_finalProject/experiments/run_reshuffel_{index}.json', 'r') as file:
        data = file.read()

    return json.loads(data)

In [ ]:
json_data_1 = read_json(1)
json_data_2 = read_json(2)
json_data_3 = read_json(3)
json_data_4 = read_json(4)
json_data_5 = read_json(5)

## Get Matching Factor from Interaction Probability

In [ ]:
grid_size = 25
neighbours = 8

def interaction_prob(x, y):
    density_prey = x / (grid_size**2)
    density_predator = y / (grid_size**2)
    prey_interact_w_predator = 1 - (1-density_predator)**neighbours
    predator_interact_w_prey = 1 - (1-density_prey)**neighbours
    interaction_prey = prey_interact_w_predator / y
    interaction_predator = predator_interact_w_prey / x
    return interaction_prey, interaction_predator

## Simulation

In [ ]:
x0 = 100#6.0
y0 = 66#3.0

Nt = 1000
tmax = 200.0
t = np.linspace(0.0,tmax, Nt)
X0 = [x0, y0]

prey_birth_rate = 0.7
predator_death_rate = 0.3

interaction_prey, interaction_predator = 8/625, 1/625#interaction_prob(x0, y0)

def lotka_volterra(X, t, alpha, beta, delta, gamma):#, machting_factor):
    x, y = X
    dotx = x * alpha - beta * x * y * interaction_predator
    doty = delta * x * y * interaction_prey - gamma * y
    return np.array([dotx, doty])

for i in range(1,2):
    json_data = {}
    count = 0
    for prey_death_rate in np.arange(0, 1.05, 0.05):
        prey_death_rate = round(prey_death_rate, 2)
        for predator_birth_rate in np.arange(0, 1.05, 0.05):
            count = count + 1
            predator_birth_rate = round(predator_birth_rate, 2)
            #matching_factor = (json_data_1[str(count)]['interaction_mean'] + json_data_2[str(count)]['interaction_mean'] + json_data_3[str(count)]['interaction_mean'] +
            #                   json_data_4[str(count)]['interaction_mean'] + json_data_5[str(count)]['interaction_mean']) / 5
            res = integrate.odeint(lotka_volterra, X0, t, args = (prey_birth_rate, prey_death_rate, predator_birth_rate, predator_death_rate))#, matching_factor))
            prey_history, predator_history = res.T
            current_data = {'prey_death_rate': prey_death_rate,
                            'predator_birth_rate': predator_birth_rate,
                            'prey_history': prey_history.tolist(),
                            'predator_history': predator_history.tolist(),
                            }
            json_data[str(count)] = current_data
            with open(f"/content/drive/MyDrive/Colab Notebooks/run_lv_independent_1_9fieldsize.json", "w") as json_file:
                json.dump(json_data, json_file, indent=4)
        print(f'-- FINISHED Prey_Death_rate {prey_death_rate} --')
    print(f'-- FINISHED Iteration {i} --')

/usr/local/lib/python3.10/dist-packages/scipy/integrate/_odepack_py.py:248: ODEintWarning: Excess work done on this call (perhaps wrong Dfun type). Run with full_output = 1 to get quantitative information.
  warnings.warn(warning_msg, ODEintWarning)


-- FINISHED Prey_Death_rate 0.0 --
-- FINISHED Prey_Death_rate 0.05 --
-- FINISHED Prey_Death_rate 0.1 --
-- FINISHED Prey_Death_rate 0.15 --
-- FINISHED Prey_Death_rate 0.2 --
-- FINISHED Prey_Death_rate 0.25 --
-- FINISHED Prey_Death_rate 0.3 --
-- FINISHED Prey_Death_rate 0.35 --
-- FINISHED Prey_Death_rate 0.4 --


# Sandbox (not used Code)

## No Matching Factor

In [ ]:
def lotka_volterra(X, t, alpha, beta, delta, gamma):
    x, y = X
    dotx = x * alpha - beta * x * y
    doty = delta * x * y - gamma * y
    return np.array([dotx, doty])

In [ ]:
x0 = 100.0
y0 = 66.0

Nt = 1000
tmax = 100.0
t = np.linspace(0.0,tmax, Nt)
X0 = [x0, y0]

prey_birth_rate = 0.7
predator_death_rate = 0.3

for i in range(1,6):
    json_data = {}
    count = 0
    for prey_death_rate in np.arange(0, 1.05, 0.05):
        prey_death_rate = round(prey_death_rate, 2)
        for predator_birth_rate in np.arange(0, 1.05, 0.05):
            count = count + 1
            predator_birth_rate = round(predator_birth_rate, 2)
            res = integrate.odeint(lotka_volterra, X0, t, args = (prey_birth_rate, prey_death_rate, predator_birth_rate, predator_death_rate))
            prey_history, predator_history = res.T
            current_data = {'prey_death_rate': prey_death_rate,
                            'predator_birth_rate': predator_birth_rate,
                            'prey_history': prey_history.tolist(),
                            'predator_history': predator_history.tolist(),
                            }
            json_data[str(count)] = current_data
            with open(f"/content/drive/MyDrive/NaCo_finalProject/experiments/run_lv_nomatching_{i}.json", "w") as json_file:
                json.dump(json_data, json_file, indent=4)
        print(f'-- FINISHED Prey_Death_rate {prey_death_rate} --')
    print(f'-- FINISHED Iteration {i} --')

<ipython-input-4-1dd335a388b2>:4: RuntimeWarning: overflow encountered in scalar multiply
  doty = delta * x * y - gamma * y
<ipython-input-4-1dd335a388b2>:3: RuntimeWarning: invalid value encountered in scalar multiply
  dotx = x * alpha - beta * x * y
<ipython-input-4-1dd335a388b2>:4: RuntimeWarning: invalid value encountered in scalar subtract
  doty = delta * x * y - gamma * y
/usr/local/lib/python3.10/dist-packages/scipy/integrate/_odepack_py.py:248: ODEintWarning: Excess accuracy requested (tolerances too small). Run with full_output = 1 to get quantitative information.
  warnings.warn(warning_msg, ODEintWarning)


-- FINISHED Prey_Death_rate 0.0 --
-- FINISHED Prey_Death_rate 0.05 --
-- FINISHED Prey_Death_rate 0.1 --
-- FINISHED Prey_Death_rate 0.15 --
-- FINISHED Prey_Death_rate 0.2 --
-- FINISHED Prey_Death_rate 0.25 --
-- FINISHED Prey_Death_rate 0.3 --
-- FINISHED Prey_Death_rate 0.35 --
-- FINISHED Prey_Death_rate 0.4 --
-- FINISHED Prey_Death_rate 0.45 --
-- FINISHED Prey_Death_rate 0.5 --
-- FINISHED Prey_Death_rate 0.55 --
-- FINISHED Prey_Death_rate 0.6 --
-- FINISHED Prey_Death_rate 0.65 --
-- FINISHED Prey_Death_rate 0.7 --
-- FINISHED Prey_Death_rate 0.75 --
-- FINISHED Prey_Death_rate 0.8 --
-- FINISHED Prey_Death_rate 0.85 --
-- FINISHED Prey_Death_rate 0.9 --
-- FINISHED Prey_Death_rate 0.95 --
-- FINISHED Prey_Death_rate 1.0 --
-- FINISHED Iteration 1 --
-- FINISHED Prey_Death_rate 0.0 --
-- FINISHED Prey_Death_rate 0.05 --
-- FINISHED Prey_Death_rate 0.1 --
-- FINISHED Prey_Death_rate 0.15 --
-- FINISHED Prey_Death_rate 0.2 --
-- FINISHED Prey_Death_rate 0.25 --
-- FINISHED Pre